# Importing the training data

In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

I'm going to try to work with the training set. I will also try to validate my data based off the training set as well and make predictions based on the test set. 

In [71]:
training = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

combine= [training, test]
all_data = pd.concat([training,test])

%matplotlib inline 
all_data.columns

In [72]:
training.info()

In [73]:
training.describe()

In [74]:
training.describe().columns

# Read a tip from Kaggle user "KenJee" that it's a good idea to seperate data into numerical variables and categorical variables to better understand the data

In [75]:
df_num = training[['Age', 'SibSp', 'Parch', 'Fare']]
df_cat = training[['Survived', 'Pclass', 'Sex', 'Ticket', 'Cabin', 'Embarked']]

### distributing the numberic variables and plotting them

In [76]:
import matplotlib.pyplot as plt

for i in df_num.columns: 
    plt.hist(df_num[i])
    plt.title(i)
    plt.show()

### We see that the age range distribution is normal but the rest is not really normal.

In [77]:
import seaborn as sns

print(df_num.corr())
sns.heatmap(df_num.corr())

### Above we can see the correlation between the different types of data, for example, age and siblings have a very LOW correlation

In [78]:
pd.pivot_table(training, index = 'Survived', values = ['Age', 'SibSp', 'Parch', 'Fare'])

In [79]:
for i in df_cat.columns: 
    sns.barplot(df_cat[i].value_counts().index, df_cat[i].value_counts()).set_title(i)
    plt.show()

### Now let's do the same method but for the categorical data

In [80]:
print(pd.pivot_table(training, index = 'Survived', columns = 'Pclass', values = 'Ticket', aggfunc = 'count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Sex', values = 'Ticket', aggfunc = 'count'))
print()
print(pd.pivot_table(training, index = 'Survived', columns = 'Embarked', values = 'Ticket', aggfunc = 'count'))
print()

In [81]:
# check missing values in training data
training.isnull().sum()


### It looks like we're missing a lot of data in age and cabin

In [82]:
# percent of missing "Age" 
print('Percent of missing "Age" records is %.2f%%' %((training['Age'].isnull().sum()/training.shape[0])*100))

In [83]:
# mean age
print('The mean of "Age" is %.2f' %(training["Age"].mean(skipna=True)))
# median age
print('The median of "Age" is %.2f' %(training["Age"].median(skipna=True)))

In [84]:
# percent of missing "cabin" 
print('Percent of missing "Cabin" records is %.2f%%' %((training['Cabin'].isnull().sum()/training.shape[0])*100))

### I'm going to fill in the data that is missing. Code from YALDA JAFARI. She also went through the same problem I did

In [88]:
#deciding about age coloumn which have almost 19% missing datas.
#Imputer age coloumn
from sklearn.impute import SimpleImputer

#train data             
Imp=SimpleImputer(strategy='median')
new_train=Imp.fit_transform(training.Age.values.reshape(-1,1))
training['Age2'] = new_train

#test data
new_test=Imp.fit_transform(test.Age.values.reshape(-1,1))
test['Age2'] = new_test


training.drop('Age',axis=1,inplace=True)
test.drop('Age',axis=1,inplace=True)


training.head()

In [89]:
training.isnull().sum()

In [93]:
training["Embarked"].value_counts()

In [98]:
# filling in the missing null values with "s" for southampton
training["Embarked"].fillna("s",inplace=True)

In [100]:
training.isnull().sum()

### Name feature

In [103]:
training["Name"]

### Putting the names into an array

In [104]:
training['Title'] =training['Name'].apply(lambda x: x.split(', ')[1].split('. ')[0].strip())
test['Title'] =training['Name'].apply(lambda x: x.split(', ')[1].split('. ')[0].strip())
training["Title"].unique()

In [ ]:
#Split the dataset to train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)